In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator 
import os
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, BatchNormalization
from keras.optimizers import Adam
from keras.layers import Activation
from os import listdir
from numpy import asarray,save
from keras.utils import load_img, img_to_array
import random
from keras import backend as K

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train = ImageDataGenerator(rescale =1.0/255.0, rotation_range=25, width_shift_range=0.1,
                         height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,vertical_flip=True,
                         horizontal_flip=True, fill_mode="nearest")
test = ImageDataGenerator(rescale=1.0/255.0, rotation_range=25, width_shift_range=0.1,
                         height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,vertical_flip=True,
                         horizontal_flip=True, fill_mode="nearest")

In [ ]:
# initial parameters
epochs = 100
lr = 1e-3
batch_size = 64
img_dims = (96,96,3)

In [ ]:
train_data = train.flow_from_directory('/content/drive/MyDrive/Gender and Age Detection Final Project Data/Gender Dataset Main Final Project/train', target_size=(96,96), class_mode='categorical')
test_data = test.flow_from_directory('/content/drive/MyDrive/Gender and Age Detection Final Project Data/Gender Dataset Main Final Project/test', target_size=(96,96), class_mode='categorical')
print('train_index :', train_data.class_indices)
print('test_index:', test_data.class_indices)

Found 5645 images belonging to 2 classes.
Found 346 images belonging to 2 classes.
train_index : {'man': 0, 'woman': 1}
test_index: {'man': 0, 'woman': 1}


In [ ]:
model= Sequential()
chanDim = -1
if K.image_data_format() == "channels_first": #Returns a string, either 'channels_first' or 'channels_last'
    chanDim = 1
#CNN1
model.add(Conv2D(32, kernel_size=(3,3), activation='relu',input_shape=(96,96,3), padding='same'))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(3,3), padding='same'))
model.add(Dropout(0.25))
#CNN2
model.add(Conv2D(64,(3,3),activation='relu', padding='same'))
model.add(BatchNormalization(axis=chanDim))
#CNN3
model.add(Conv2D(128,(3,3),activation='relu', padding='same'))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
model.add(Dropout(0.25))  
#CNN4
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', padding="same"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
model.add(Dropout(0.25))

batch_size=128
epochs =100
classes = 2

In [ ]:
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(classes))
model.add(Activation("sigmoid"))

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 96, 96, 32)        896       
                                                                 
 batch_normalization_20 (Bat  (None, 96, 96, 32)       128       
 chNormalization)                                                
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 dropout_17 (Dropout)        (None, 32, 32, 32)        0         
                                                                 
 conv2d_13 (Conv2D)          (None, 32, 32, 64)        18496     
                                                                 
 batch_normalization_21 (Bat  (None, 32, 32, 64)      

In [ ]:
from keras.optimizers import Adam
opt = Adam(learning_rate=lr, decay=lr/epochs)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 96, 96, 32)        896       
                                                                 
 batch_normalization_20 (Bat  (None, 96, 96, 32)       128       
 chNormalization)                                                
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 dropout_17 (Dropout)        (None, 32, 32, 32)        0         
                                                                 
 conv2d_13 (Conv2D)          (None, 32, 32, 64)        18496     
                                                                 
 batch_normalization_21 (Bat  (None, 32, 32, 64)      

In [ ]:
his=model.fit(train_data,epochs=epochs,batch_size=128,verbose=1,validation_data=test_data)
value=model.evaluate(train_data,verbose=0)
print('loss', value[0])
print('accuracy', value[1])

Epoch 1/100
177/177 [==============================] - 1400s 8s/step - loss: 0.7609 - accuracy: 0.6677 - val_loss: 1.2326 - val_accuracy: 0.5087
Epoch 2/100
177/177 [==============================] - 30s 172ms/step - loss: 0.5995 - accuracy: 0.7268 - val_loss: 2.0239 - val_accuracy: 0.5260
Epoch 3/100
177/177 [==============================] - 31s 175ms/step - loss: 0.5723 - accuracy: 0.7364 - val_loss: 1.1571 - val_accuracy: 0.6301
Epoch 4/100
177/177 [==============================] - 30s 171ms/step - loss: 0.5243 - accuracy: 0.7605 - val_loss: 0.4024 - val_accuracy: 0.7803
Epoch 5/100
177/177 [==============================] - 30s 170ms/step - loss: 0.4940 - accuracy: 0.7708 - val_loss: 1.0654 - val_accuracy: 0.5202
Epoch 6/100
177/177 [==============================] - 31s 173ms/step - loss: 0.4795 - accuracy: 0.7860 - val_loss: 0.5571 - val_accuracy: 0.7197
Epoch 7/100
177/177 [==============================] - 30s 168ms/step - loss: 0.4433 - accuracy: 0.8005 - val_loss: 0.5584 - 

In [ ]:
model.save('/content/drive/MyDrive/Model Save/Gender_Detection.h5')